In [2]:
import cifar10
from cifar10 import num_classes
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [3]:
train_data,train_labels,onehot_train=cifar10.load_training_data()
test_data,test_labels,onehot_test=cifar10.load_test_data()
train_data=train_data.astype('float32')
train_labels=train_labels.astype('int32')
test_data=test_data.astype('float32')
test_labels=test_labels.astype('int32')

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_1'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_2'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_3'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_4'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_5'>
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/test_batch'>


In [4]:
test_labels.shape

(10000,)

In [5]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 32, 32,3])
    
    
    
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv1_bn = tf.layers.batch_normalization(
        inputs=conv1,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
    )
    conv1_bn_relu = tf.nn.relu(conv1_bn)
    
    conv2 = tf.layers.conv2d(
      inputs=conv1_bn_relu,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv2_bn = tf.layers.batch_normalization(
        inputs=conv2,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv2_bn_relu = tf.nn.relu(conv2_bn)
    pool1 = tf.layers.max_pooling2d(inputs=conv2_bn_relu, pool_size=[2, 2], strides=2)
    
    dropout1 = tf.layers.dropout(
      inputs=pool1, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)


    
    
    conv3 = tf.layers.conv2d(
      inputs=dropout1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv3_bn = tf.layers.batch_normalization(
        inputs=conv3,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
    )
    conv3_bn_relu = tf.nn.relu(conv3_bn)
    
    conv4 = tf.layers.conv2d(
      inputs=conv3_bn_relu,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv4_bn = tf.layers.batch_normalization(
        inputs=conv4,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv4_bn_relu = tf.nn.relu(conv4_bn)
    pool2 = tf.layers.max_pooling2d(inputs=conv4_bn_relu, pool_size=[2, 2], strides=2)
    
    
    dropout2 = tf.layers.dropout(
      inputs=pool2, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

    
    conv5 = tf.layers.conv2d(
      inputs=dropout2,
      filters=128,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv5_bn = tf.layers.batch_normalization(
        inputs=conv5,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
    )
    conv5_bn_relu = tf.nn.relu(conv5_bn)
    
    conv6 = tf.layers.conv2d(
      inputs=conv5_bn_relu,
      filters=128,
      kernel_size=[3, 3],
      padding="same",
      activation=None)
    conv6_bn = tf.layers.batch_normalization(
        inputs=conv6,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
    )
    conv6_bn_relu = tf.nn.relu(conv6_bn)
    pool3 = tf.layers.max_pooling2d(inputs=conv6_bn_relu, pool_size=[2, 2], strides=2)
    
    
    dropout3 = tf.layers.dropout(
      inputs=pool3, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

    flattened = tf.reshape(dropout3, [-1, 4 * 4 * 128])
    dense1 = tf.layers.dense(inputs=flattened, units=2048, activation=None)
    logits = tf.layers.dense(inputs=dense1, units=10)

    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits/temperature, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




In [6]:
def cross_entropy2(tar_soft_t, pred_logits):
   pred_soft_t = tf.nn.softmax(pred_logits/temperature)
   pred_log = -1 * tf.log(pred_soft_t)
   product =tf.multiply(tar_soft_t,pred_log)
   return tf.reduce_mean(product)
    
def custom_loss(y_true, pred_logits, tar_soft_t):
  return  loss_weight * tf.losses.sparse_softmax_cross_entropy(labels = y_true, logits = pred_logits)+(1-loss_weight)*cross_entropy2(tar_soft_t,pred_logits)


In [7]:
def getFilterData(f,l):
  sess=tf.InteractiveSession()
  tf.train.start_queue_runners(sess)
  data_s=f['x'].eval()
  out_s=l.eval()
  sess.close()
  return data_s,out_s

In [8]:

def student_model_fn(features, labels, mode):

    input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
    if mode == tf.estimator.ModeKeys.TRAIN:
        data_swap,out_swap=getFilterData(features,labels)
        eval_teacher_fn = tf.estimator.inputs.numpy_input_fn(
          x={"x":data_swap},
          y=out_swap,
          batch_size=100,
          shuffle=False)
        eval_teacher=cifar_classifier.evaluate(input_fn=eval_teacher_fn)
        outlog.write('%f;' % eval_teacher['accuracy'])
        outlog.write('%f;' % eval_teacher['loss'])
        outlog.write('\n')
        predictions=cifar_classifier.predict(input_fn=eval_teacher_fn)
        teacher_pred=list(predictions)
        teacher_soft=[ p['probabilities'] for p in teacher_pred]
        temp=np.array(teacher_soft)

        teacher_soft_t=tf.convert_to_tensor(temp)




    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=2 ** no_of_fea,
        kernel_size=[kernel_s,kernel_s],
        padding="same",
        activation=None)
    conv1_bn = tf.layers.batch_normalization(
        inputs=conv1,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
        )
    conv1_bn_relu = tf.nn.relu(conv1_bn)

    conv2 = tf.layers.conv2d(
        inputs=conv1_bn_relu,
        filters=2 ** no_of_fea,
        kernel_size=[kernel_s, kernel_s],
        padding="same",
        activation=None)
    conv2_bn = tf.layers.batch_normalization(
        inputs=conv2,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        )
    conv2_bn_relu = tf.nn.relu(conv2_bn)
    pool1 = tf.layers.max_pooling2d(inputs=conv2_bn_relu, pool_size=[2, 2], strides=2)

    dropout1 = tf.layers.dropout(
        inputs=pool1, rate=0.2, training=mode == tf.estimator.ModeKeys.TRAIN)

    conv3 = tf.layers.conv2d(
        inputs=dropout1,
        filters=2 ** (no_of_fea + 1) ,
        kernel_size=[kernel_s, kernel_s],
        padding="same",
        activation=None)
    conv3_bn = tf.layers.batch_normalization(
        inputs=conv3,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
        )
    conv3_bn_relu = tf.nn.relu(conv3_bn)

    conv4 = tf.layers.conv2d(
        inputs=conv3_bn_relu,
        filters=2 ** (no_of_fea + 1),
        kernel_size=[kernel_s, kernel_s],
        padding="same",
        activation=None)
    conv4_bn = tf.layers.batch_normalization(
        inputs=conv4,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        )
    conv4_bn_relu = tf.nn.relu(conv4_bn)
    pool2 = tf.layers.max_pooling2d(inputs=conv4_bn_relu, pool_size=[2, 2], strides=2)


    dropout2 = tf.layers.dropout(
        inputs=pool2, rate=0.3, training=mode == tf.estimator.ModeKeys.TRAIN)


    conv5 = tf.layers.conv2d(
        inputs=dropout2,
        filters=2 ** (no_of_fea+2),
        kernel_size=[kernel_s, kernel_s],
        padding="same",
        activation=None)
    
    conv5_bn = tf.layers.batch_normalization(
        inputs=conv5,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True
        )
    conv5_bn_relu = tf.nn.relu(conv5_bn)

    conv6 = tf.layers.conv2d(
        inputs=conv5_bn_relu,
        filters=2 ** (no_of_fea + 2),
        kernel_size=[kernel_s, kernel_s],
        padding="same",
        activation=None)
    conv6_bn = tf.layers.batch_normalization(
        inputs=conv6,
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        )
    conv6_bn_relu = tf.nn.relu(conv6_bn)
    pool3 = tf.layers.max_pooling2d(inputs=conv6_bn_relu, pool_size=[2, 2], strides=2)


    dropout3 = tf.layers.dropout(
        inputs=pool3, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    flattened = tf.reshape(dropout3, [-1, 4 * 4 * (2 ** (no_of_fea + 2))])
    dense1 = tf.layers.dense(inputs=flattened, units=fc_u, activation=None)
    logits = tf.layers.dense(inputs=dense1, units=10)

    
    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss=custom_loss(labels, logits,teacher_soft_t)
    else:
        print(labels.shape,logits.shape)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)


    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
    }
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)




In [9]:
print(train_labels.shape)

(50000,)


In [13]:

tf.logging.set_verbosity(tf.logging.ERROR)
# Create the Estimator
cifar_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="./cifar_model_teacher_6_conv_earlystop")

temp_list=[1,2,5,10,15,20,30]
loss_weight_list=[0,0.25,0.5,0.75,1]
p=-1
l=-1
def getdatap():
    global p
    p += 1
    if p>len(train_labels)/100:
        p=0
    data=train_data[p*100:(p+1)*100]
    labels=train_labels[p*100:(p+1)*100]
    
    datatensor=tf.convert_to_tensor(data)
    labeltensor=tf.convert_to_tensor(labels)
    
    
    return {'x':datatensor},labeltensor

out = open('cifar_ensemble2.csv', 'w')
outlog = open('out_log.csv', 'w')

temp_list=[10]
loss_weight_list=[0.75]

##HyperParameters
lSize_of_kernel = [3,5]
lNo_of_Feature_Maps = [2,3,4,5]
lFC_UNIT=[32,64,128]

for i in temp_list:
    for j in loss_weight_list:
        for k in lSize_of_kernel:
            for l in lNo_of_Feature_Maps:
                 for m in lFC_UNIT:
                    # Create the Estimator
                    print(i,j,k,l,m)
                    temperature=i
                    loss_weight=j
                    kernel_s=k
                    no_of_fea=l
                    fc_u=m
                    student_classifier = tf.estimator.Estimator(
                      model_fn=student_model_fn, model_dir="./models/CIFAR_SINGLE_student_new_"+str(i)+"_w_"+str(j)+"_f_"+str(k)+"_ker_"+str(l)+"_fc_"+str(m))
                   
        
                    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
                      x={"x": train_data},
                      y=train_labels,
                      batch_size=128,
                      shuffle=False)

                    student_classifier.train(
                      input_fn=eval_train_fn,
                      steps=1000,
                      hooks=None)


                    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
                      x={"x": train_data},
                      y=train_labels,
                      shuffle=False)
                    train_result=student_classifier.evaluate(input_fn=eval_train_fn)
                    print("Train ",train_result)
                    out.write('%f;' % i)
                    out.write('%f;' % j)
                    out.write('%f;' % train_result['accuracy'])
                    out.write('%f;' % train_result['loss'])

                    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                      x={"x": test_data},
                      y=test_labels,
                      num_epochs=1,
                      shuffle=False)
                    eval_result=student_classifier.evaluate(input_fn=eval_input_fn)
                    # output=list(predictions)
                    print("Test",eval_result)
                    out.write('%f;' % eval_result['accuracy'])
                    out.write('%f;' % eval_result['loss'])
            #         acc=eval_result
            #         out.write(eval_result)
                    out.write('\n')

out.close()
outlog.close()

10 0.75 3 2 32
ERROR:tensorflow:Exception in QueueRunner: Attempted to use a closed Session.


Exception in thread Thread-367:
Traceback (most recent call last):
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\usinh\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\estimator\inputs\queues\feeding_queue_runner.py", line 94, in _run
    sess.run(enqueue_op, feed_dict=feed_dict)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 900, in run
    run_metadata_ptr)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1058, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.



InvalidArgumentError: Incompatible shapes: [128,10] vs. [80,10]
	 [[Node: Mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Const, mul_1)]]
	 [[Node: add/_873 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1022_add", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Mul', defined at:
  File "C:\Users\usinh\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\usinh\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\usinh\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\usinh\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\usinh\Anaco

In [ ]:
X=[]
for i in temp_list:
    for j in loss_weight_list:
        for k in lSize_of_kernel:
            for l in lNo_of_Feature_Maps:
                 for m in lFC_UNIT:
                   # Create the Estimator
                    print(i,j,k,l,m)
                    temperature=i
                    loss_weight=j
                    kernel_s=k
                    no_of_fea=l
                    fc_u=m
                    student_classifier = tf.estimator.Estimator(
                      model_fn=student_model_fn, model_dir="./models/CIFAR_SINGLE_student_new_"+str(i)+"_w_"+str(j)+"_f_"+str(k)+"_ker_"+str(l)+"_fc_"+str(m))
                   
        
                 
                    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                      x={"x": test_data},
                      y=test_labels,
                      num_epochs=1,
                      shuffle=False)
                    eval_results=student_classifier.predict(input_fn=eval_input_fn)
                    student_pred=list(eval_results)
                    student_classes=[ p['classes'] for p in student_pred]                    
                    X.append(student_classes)
                    

In [12]:
from scipy import stats
X = tf.stack(X)
sess = tf.InteractiveSession()
a = X.eval()
sess.close()
x = stats.mode(a)
x = x.mode
print(test_labels)
print((x == test_labels).sum()/len(test_labels))

[3 8 8 ..., 5 1 7]
0.6219
